In [2]:
import numpy as np
import matplotlib.pyplot as plt

import histlite as hl
import csky as cy
import glob as glob

%matplotlib inline

In [3]:
timer = cy.timing.Timer()
time = timer.time

In [31]:
# Reading in the signal and background files and change the keys to the on

def get_signal_files(key):
    dir = "/data/user/bschlueter/software/nusources_dataset_converters/data/queso_"+str(key)+".npy"
    sig = np.load(dir)
    sig = np.hstack(sig)
    sig_upgraded = np.lib.recfunctions.rename_fields(sig, {'ow': 'oneweight','trueE':'true_energy'})
    sig_upgraded = sig_upgraded.view(np.recarray)
    return sig_upgraded

bg_filename = "/data/user/bschlueter/software/nusources_dataset_converters/data/muongun_131028.npy"
bg = np.load(bg_filename)
bg_upgraded = np.lib.recfunctions.rename_fields(bg, {'atmo_weight': 'weight'})
bg_upgraded = bg_upgraded.view(np.recarray)

In [32]:
#loading the data and saving it in a dataset list
keys = ['120028','121028','140028','141028','160028','161028']
Dataset = []
for key in keys:  
    Dataset.append(cy.selections.Dataset(key = key,livetime=8*60*60, sig= get_signal_files(key), bg = bg_upgraded))

with time('ana setup (from scratch)'):
    ana = cy.get_analysis(cy.selections.repo,spec = Dataset)

Setting up Analysis for:

Done.

0:00:00.003542 elapsed.


In [33]:
Dataset

[Dataset('120028', livetime = 28800s) with  and 0.0006169195555999903 expected bg events and sum(ow*E^-2) = 1.60,
 Dataset('121028', livetime = 28800s) with  and 0.0006169195555999903 expected bg events and sum(ow*E^-2) = 0.41,
 Dataset('140028', livetime = 28800s) with  and 0.0006169195555999903 expected bg events and sum(ow*E^-2) = 1.46,
 Dataset('141028', livetime = 28800s) with  and 0.0006169195555999903 expected bg events and sum(ow*E^-2) = 1.47,
 Dataset('160028', livetime = 28800s) with  and 0.0006169195555999903 expected bg events and sum(ow*E^-2) = 1.02,
 Dataset('161028', livetime = 28800s) with  and 0.0006169195555999903 expected bg events and sum(ow*E^-2) = 0.50]

<span style="color:red;">Problem: empty keys in ana.</span>

In [35]:
cy.CONF['ana'] = ana
cy.CONF['mp_cpus'] = 1
print(ana)

Analysis(keys=[])


- I'm not exactly sure what config I should use for the BG estimation
  